Created on Wednesday 13 January 2021

**Group 5 - Classification**  
**Extraction features syntaxe**

@authors : Jeremy Johann

The notebook that contain all the code. You just need to put in the fonction the art_id, the content with lemmatization, the title of the article and the Bag Of Word (BOW). This code train, save all the model and give the prediction.

# Pip install

In [ ]:
pip install scikit-learn-extra

     |████████████████████████████████| 624kB 4.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.1.0b2-cp36-cp36m-linux_x86_64.whl size=339584 sha256=f920b1685b0f32eee0c8c751a201a504fb0dbf32aee082dfa1fcac9f9f759183
  Stored in directory: /root/.cache/pip/wheels/04/01/0f/943bffb48bac048fa216b4325f1a6c939491ccb0ff500e08f4
Successfully built scikit-learn-extra


# Import libraries

In [ ]:
import re
import os
import nltk
import math
import string
import pickle
import warnings
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook
from tqdm import tqdm_notebook as tqdm

from xgboost import XGBClassifier

from scipy.spatial import distance

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn import svm
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn_extra.cluster import KMedoids
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, silhouette_score, calinski_harabasz_score
from sklearn.ensemble import BaggingClassifier, IsolationForest, RandomForestClassifier, ExtraTreesClassifier

warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Create link between drive and notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Detection documents innovants et gamme de gestion

In [ ]:
# Counts the number of words
def nb_word(text: list) -> int:
    """Documentation
    Parameters:
        text: texts of the article

    Out (if exists):
        nb_word: number of word in  the document
    """
    nb_words: list = []
    nb: int = 0

    # Browse through the different texts
    for i in text:
        # Removes special characters
        i.replace(',', ' ')
        i.replace('.', ' ')
        i.replace('!', ' ')
        i.replace('?', ' ')
        i.replace('/', ' ')
        # Creates a list with all the words present in the text
        list_words: list = i.split()
        # Counts the number of words present in the text
        nb_words.append(len(list_words))

    return nb_words

In [ ]:
# This function calculate the numbers of key words given by a list in the documents
def count_key_words(data: list, l: list) -> list:
    """Documentation
    Parameters:
        data: List of articles
        l: List of word that we will check in the sentences

    Out:
        res: List where each value is the number of time where key word appear in the article
    """
    data = data.tolist()
    list_mot_unique=[]
    list_mot_compose=[]

    # Count the number of time where the words in the list appear
    for elem in l :
      cpt = 0
      for car in elem :
        if (car == " ") :
          cpt+=1
      if cpt==0 :
        list_mot_unique.append(elem)
      else :
        list_mot_compose.append(elem)
    res: list = []

    for i in range(len(data)):
      sentence: str = data[i]
      if sentence is None:
        res.append(0)
      else:
        sentence = sentence.lower()
        sentence = sentence.split()
        t: int = 0
        for j in sentence:
          if (j in list(list_mot_unique)):
            t = t + 1
        sentence: str = data[i]
        for elem in list_mot_compose:
          if elem in sentence:
            t = t + 1
      res.append(t)

    return res

In [ ]:
# This function calculate the numbers of sentences in each documents
def phrases(data: pd.DataFrame, col: str) -> list:
    """Documentation
    Parameters:
        data: Dataframe with all the data
        columns: The columns of the dataframe that we will use

    Out:
        l: List where each value is the number of sentence in a article
    """
    l: list = []

    # Count the number of sentence
    for i in range(len(data[col])):
        sentences: str = data[col].tolist()[i]

        if not isinstance(sentences, str):
            sentences: str = str(sentences)

        if (sentences is None):
            count_sentence.append(0)
        else:
            sentences = sentences.replace("..", ".")
            sentences = sentences.replace("...", ".")
            sentences = sentences.replace("!", ".")
            sentences = sentences.replace("!!", ".")
            sentences = sentences.replace("!!!", ".")
            sentences = sentences.replace("?", ".")
            sentences = sentences.replace("??", ".")
            sentences = sentences.replace("???", ".")
            sentences = sentences.replace("?!", ".")
            sentences = sentences.replace("!?", ".")
            l.append(len(sent_tokenize(sentences)))

    return l

In [ ]:
# count the number of distinct words in each documents
def count_words_diff(df: pd.DataFrame, list_key : list) -> int :
  """Documentation
  Parameter:
      texte: text of an article
      
  Out:
      len(dico): lenght of a dictionary
  """
  result = []
  for j in tqdm(range(len(df))):
    liste = df[j].split()
    dico = {}
    fait = False
    for i in range(len(liste)) :
      try : 
        if (liste[i]+' '+liste[i+1]+' '+liste[i+2]) in list_key :
          dic(liste[i]+' '+liste[i+1]+' '+liste[i+2],dico)
          fait = True
        else :
          try :
            if (liste[i]+' '+liste[i+1]) in list_key :
              dic(liste[i]+' '+liste[i+1],dico)
              fait = True
          except :
            pass
      except :
        pass
      if not fait :
        dic(liste[i],dico)
      fait = False
    result.append(len(dico))

  return result

In [ ]:
# Return a dico with all the distinct word and their numbers of appearances
def comparaison_words_diff(texte: str, list_key : list) -> int :
  """Documentation
    Parameter:
        texte: text of an article

    Out:
        len(dico): lenght of a dictionary
  """
  liste = texte.split()
  dico = {}
  fait = False

  for i in range(len(liste)) :
    try : 
      if (liste[i]+' '+liste[i+1]+' '+liste[i+2]) in list_key :
        dic(liste[i]+' '+liste[i+1]+' '+liste[i+2],dico)
        fait = True
      else :
        try :
          if (liste[i]+' '+liste[i+1]) in list_key :
            dic(liste[i]+' '+liste[i+1],dico)
            fait = True
        except :
          pass
    except :
      pass
    if not fait :
      dic(liste[i],dico)
    fait = False
    
  return dico

In [ ]:
# Count the distinct word of a list
def count_key_words_diff(liste : list) -> int:
  """Documentation
  Parameter:
      liste: list of key words

  Out:
      len(dico): lenght of a dictionary
  """
  dico = {}
  for i in range(len(liste)) :
    dic(liste[i],dico)

  return (dico)

In [ ]:
# Look up a word in a dictionary
def dic(term: str,dico: dict) :
  """Documentation
  Parameter:
      term: one or set of words
      dico: dictionary
  """
  if term in dico.keys() :
    dico[term] += 1
  elif term != '' :
    dico[term] = 1

In [ ]:
# count the number of distinct key words of a list present in each text
def key_word_in_doc(df : pd.DataFrame, list_key : list):
  """Documentation
    Parameter:
        df: Column of dataframe who contains all text
        list_key: List of keys word
  """
  key_word = []
  for i in tqdm(range(len(df))):
    sortie =  comparaison_words_diff(df[i], list_key)
    liste_cle = []
    for cle in sortie.keys():
      liste_cle.append(cle)
    tot =0
    for i in liste_cle:
      if i in (list_key.tolist()):
        tot = tot + 1
    key_word.append(tot)

  return (key_word)

In [ ]:
# Use to generate weighted randomness to say if a document talk about management range or not
def score_to_threshold(x: int) -> int:
    """Documentation
    Parameters:
        x: Innovation score associated with an article

    Out:
        Threshold probability used to differentiate innovative from non-innovative documents
    """

    return np.arctan(x * 100) / np.pi * 2 * 0.2 + 0.8


# Function to tell if a document talk about innovation or not 
def innovation(data_nb1: pd.DataFrame, data_nb2: pd.DataFrame, data_nb3: pd.DataFrame, data_ratio1: pd.DataFrame, data_ratio2: pd.DataFrame, data_ratio3 : pd.DataFrame) -> list:
  """Documentation
    Parameters:
          data_nb1: First column of a dataframe who describe a number of key words presents in text
          data_nb2: Second column of a dataframe who describe a number of key words presents in title
          data_nb3: Third column who describe the distinct number of key word presents in text
          data_ratio1: Linked column of data_nb1 who represents a ratio of key word apparition in text
          compared to the total number of words in text
          data_ratio2: Linked column of data_nb2 who represents a ratio of key word apparition in title
          compared to the total number of words in title
          data_ratio3: Linked column of data_nb3 who represents a ratio of distinct key word apparition in text
          compared to the total number of distinct word in text

     Out:
          list: Represents score of innovation, calculate in the function innovation
  """
  res: list = []
  seuil: int = 0.18

  for i in tqdm(range(len(data_nb1))):
    valeur = 0.5*(data_nb1[i]*data_ratio1[i]) + 0.5*(data_nb2[i]*data_ratio2[i]) + (data_nb3[i]*data_ratio3[i])
    alea = np.random.random()
    if valeur > seuil:
        res.append(1)
    elif (valeur < seuil) & (data_nb1[i] == 0) & (data_nb2[i] == 0):
        res.append(0)
    elif (valeur < seuil) & (data_ratio1[i] < 0.0002) & (data_ratio2[i] < 0.0005):
        res.append(0)
    else:
        if valeur > seuil / 2 and alea > score_to_threshold((seuil - valeur)):
            res.append(1)
        elif valeur < seuil / 2 and alea > score_to_threshold(valeur):
            res.append(0)
        else:
            res.append('?')

  return res

In [ ]:
# Function to tell if a document talk about management range or not
def gestion(data_nb1: pd.DataFrame, data_nb2: pd.DataFrame, data_nb3: pd.DataFrame, data_ratio1: pd.DataFrame, data_ratio2: pd.DataFrame, data_ratio3 : pd.DataFrame) -> list:
    """Documentation
    this function calculate a score who represents if a document talk about management range or not.
    The higher score show document who talk about management range and the weakest show document who don't talk about management range.
    We define a threshold and the document who have a score higher of this threshold take a 1 who represent management range.
    Documents whith little score are consider such as they don't talk about management range take 0.
    The others have a '?' because we don't know if they talk about management range or not and we try in the rest of notebooks to create labels
    for the documents with '?' thanks to documents who have labels
    Parameters:
          data_nb1: First column of a dataframe who describe a number of key words presents in text
          data_nb2: Second column of a dataframe who describe a number of key words presents in title
          data_nb3: Third column who describe the distinct number of key word presents in text
          data_ratio1: Linked column of data_nb1 who represents a ratio of key word apparition in text
          compared to the total number of words in text
          data_ratio2: Linked column of data_nb2 who represents a ratio of key word apparition in title
          compared to the total number of words in title
          data_ratio3: Linked column of data_nb3 who represents a ratio of distinct key word apparition in text
          compared to the total number of distinct word in text

    Out:
          list: represents whether a document is innovative or not (i.e. yes = 1, no = 0, don't know = ?)
    """
  res: list = []
  seuil: int = 0.70

  for i in tqdm(range(len(data_nb1))):
    valeur = 0.5*(data_nb1[i]*data_ratio1[i]) + 0.5*(data_nb2[i]*data_ratio2[i]) + (data_nb3[i]*data_ratio3[i])
    alea = np.random.random()
    if valeur > seuil:
        res.append(1)
    elif (valeur < seuil) & (data_nb1[i] == 0) & (data_nb2[i] == 0):
        res.append(0)
    elif (valeur < seuil) & (data_ratio1[i] < 0.0002) & (data_ratio2[i] < 0.0005):
        res.append(0)
    else:
        if valeur > seuil / 2 and alea > score_to_threshold((seuil - valeur)):
            res.append(1)
        elif valeur < seuil / 2 and alea > score_to_threshold(valeur):
            res.append(0)
        else:
            res.append('?')

  return res

In [ ]:
# Function use to create all features we need
def create_features(df : pd.DataFrame, texte : str, title : str, inno_ges : str, df_lexique : pd.DataFrame) -> pd.DataFrame:
  """Documentation
    Parameter:
        df: A dataframe on which we will create features
        texte : Name of column who contain text
        title : Name of column who contain title of article
        inno_ges : String to specify if we want an analyse on management range or innovation.
        We can only choose options "innovation" or "gestion"
  """
  df["nb_key_words"]: pd.DataFrame = count_key_words(df[texte], df_lexique["key_words_lemma"])
  df["nb_key_words_title"]: pd.DataFrame = count_key_words(df[title], df_lexique["key_words_lemma"])
  df["nb_words"]: pd.DataFrame = nb_word(df[texte])
  df["nb_words_title"]: pd.DataFrame = nb_word(df[title])
  df["nb_sentences"]: pd.DataFrame  = phrases(df, texte)
  df["average_word_sentence"]: pd.DataFrame = df["nb_words"] / df["nb_sentences"]
  df["ratio_word_title_on_word"]: pd.DataFrame  = df["nb_words_title"] / df["nb_words"]
  df['ratio_key_words']: pd.DataFrame  = df['nb_key_words']/df['nb_words']
  df['ratio_key_words']: pd.DataFrame  = df['ratio_key_words'].fillna(0)
  df['ratio_key_sentences']: pd.DataFrame  = df['nb_key_words']/df['nb_sentences']
  df['ratio_key_sentences']: pd.DataFrame  = df['ratio_key_sentences'].fillna(0)
  df['ratio_key_word_title']: pd.DataFrame  = df['nb_key_words_title'] / df['nb_words_title']
  df['ratio_key_word_title']: pd.DataFrame  = df['ratio_key_word_title'].fillna(0)
  df['word_key_diff']: pd.DataFrame = key_word_in_doc(df[texte].tolist(), df_lexique['key_words_lemma'])
  df['word_diff']: pd.DataFrame = count_words_diff(df[texte].tolist(), df_lexique['key_words_lemma'])
  df['ratio_key_word_diff']: pd.DataFrame  = df['word_key_diff'] / df['word_diff']
  
  return df

In [ ]:
# Function use for create a label for each documents
def create_label(df : pd.DataFrame, inno_ges : str, df_lexique : pd.DataFrame) -> pd.DataFrame:
  """Documentation
    Parameter:
        df: A dataframe who contains article that we are going to classify
        inno_ges: String to specify if we want an analyse on management range or innovation.
        We can only choose options "innovation" or "gestion"
        df_lexique : A daframe with the lexique of gestion or innovation
  """
  df = create_features(df, "art_content_clean_without_lem", "art_title", inno_ges, df_lexique)

  if (inno_ges == 'innovation'):
    df['prediction_supervise'] = innovation(df['nb_key_words'].tolist(),df['nb_key_words_title'].tolist(),df['word_key_diff'].tolist(), df['ratio_key_words'].tolist(),df['ratio_key_word_title'].tolist(), df['ratio_key_word_diff'].tolist())
  elif (inno_ges == 'gestion'):
    df['prediction_supervise'] = gestion(df['nb_key_words'].tolist(),df['nb_key_words_title'].tolist(),df['word_key_diff'].tolist(), df['ratio_key_words'].tolist(),df['ratio_key_word_title'].tolist(), df['ratio_key_word_diff'].tolist())

  var_utile = df.drop(["art_id","art_content_clean_without_lem","art_title","prediction_supervise"], axis = 1)
  var_utile = var_utile.drop(["nb_key_words","nb_key_words_title","nb_words","nb_words_title","nb_sentences","average_word_sentence","ratio_word_title_on_word",'ratio_key_words','ratio_key_sentences','ratio_key_word_title','word_key_diff','word_diff','ratio_key_word_diff'],axis = 1)

  var_utile = pd.concat([var_utile, df['prediction_supervise']], axis = 1)

  tout = var_utile[var_utile['prediction_supervise'] != '?']
  unlabeled = var_utile[var_utile['prediction_supervise'] == '?']

  X_train = tout.drop('prediction_supervise', axis=1)
  y_train = tout.prediction_supervise
  y_train = pd.to_numeric(y_train)

  X_unlabeled = unlabeled.drop('prediction_supervise', axis=1)

  if (inno_ges == 'innovation'):
    model1 = XGBClassifier(random_state=0)
    model2 = DecisionTreeClassifier(random_state=0)
    model3 = LogisticRegression(penalty = 'l1', solver = 'liblinear',random_state=0)
  elif (inno_ges == 'gestion'):
    model1 = XGBClassifier(random_state=0)
    model2 = LogisticRegression(penalty = 'l1', solver = 'liblinear',random_state=0)
    model3 = svm.SVC(C = 4,kernel='linear', probability= True,random_state=0)
  # Initiate iteration counter
  iterations = 0

  # Containers to hold f1_scores and # of pseudo-labels
  train_f1s = []
  pseudo_labels = []

  # Assign value to initiate while loop
  high_prob = [1] 

  # Loop will run until there are no more high-probability pseudo-labels
  while len(high_prob) > 0:
        
    # Fit classifier and make train/test predictions
    model1.fit(X_train, y_train)
    y_hat_train = model1.predict(X_train)

    # Calculate and print iteration # and f1 scores, and store f1 scores
    train_f1 = f1_score(y_train, y_hat_train)
    print(f"Iteration {iterations}")
    print(f"Train f1: {train_f1}")
    train_f1s.append(train_f1)
   
    if (len(X_unlabeled) > 0):
      # Generate predictions and probabilities for unlabeled data
      print(f"Now predicting labels for unlabeled data...")

      pred_probs : np.ndarray = model1.predict_proba(X_unlabeled)
      preds : np.ndarray = model1.predict(X_unlabeled)
      prob_0 : list = pred_probs[:,0]
      prob_1 : list = pred_probs[:,1]

      # Store predictions and probabilities in dataframe
      df_pred_prob : pd.DataFrame = pd.DataFrame([])
      df_pred_prob['preds'] = preds
      df_pred_prob['prob_0'] = prob_0
      df_pred_prob['prob_1'] = prob_1
      df_pred_prob.index = X_unlabeled.index
    
      # Separate predictions with > 99% probability
      high_prob : pd.DataFrame = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.99],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > 0.99]],
                          axis=0)
      print(f"{len(high_prob)} high-probability predictions added to training data.")
    
      pseudo_labels.append(len(high_prob))

      # Add pseudo-labeled data to training data
      X_train = pd.concat([X_train, X_unlabeled.loc[high_prob.index]], axis=0)
      y_train = pd.concat([y_train, high_prob.preds])      
    
      # Drop pseudo-labeled instances from unlabeled data
      X_unlabeled = X_unlabeled.drop(index=high_prob.index)
      print(f"{len(X_unlabeled)} unlabeled instances remaining.\n")
    
      # Update iteration counter
      iterations += 1
    else :
      high_prob = []
      print(f'end of process.')

  # Initiate iteration counter
  iterations = 0

  # Containers to hold f1_scores and # of pseudo-labels
  train_f1s = []
  pseudo_labels = []

  # Assign value to initiate while loop
  high_prob = [1] 

  # Loop will run until there are no more high-probability pseudo-labels
  while len(high_prob) > 0:
        
    # Fit classifier and make train/test predictions
    model2.fit(X_train, y_train)
    y_hat_train = model2.predict(X_train)

    # Calculate and print iteration # and f1 scores, and store f1 scores
    train_f1 = f1_score(y_train, y_hat_train)
    print(f"Iteration {iterations}")
    print(f"Train f1: {train_f1}")
    train_f1s.append(train_f1)
   
    if (len(X_unlabeled) > 0):
      # Generate predictions and probabilities for unlabeled data
      print(f"Now predicting labels for unlabeled data...")

      pred_probs : np.ndarray = model2.predict_proba(X_unlabeled)
      preds : np.ndarray = model2.predict(X_unlabeled)
      prob_0 : list = pred_probs[:,0]
      prob_1 : list = pred_probs[:,1]

      # Store predictions and probabilities in dataframe
      df_pred_prob : pd.DataFrame = pd.DataFrame([])
      df_pred_prob['preds'] = preds
      df_pred_prob['prob_0'] = prob_0
      df_pred_prob['prob_1'] = prob_1
      df_pred_prob.index = X_unlabeled.index
    
      # Separate predictions with > 99% probability
      high_prob : pd.DataFrame = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.99],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > 0.99]],
                          axis=0)
      print(f"{len(high_prob)} high-probability predictions added to training data.")
    
      pseudo_labels.append(len(high_prob))

      # Add pseudo-labeled data to training data
      X_train = pd.concat([X_train, X_unlabeled.loc[high_prob.index]], axis=0)
      y_train = pd.concat([y_train, high_prob.preds])      
    
      # Drop pseudo-labeled instances from unlabeled data
      X_unlabeled = X_unlabeled.drop(index=high_prob.index)
      print(f"{len(X_unlabeled)} unlabeled instances remaining.\n")
    
      # Update iteration counter
      iterations += 1
    else :
      high_prob = []
      print(f'end of process.')

  # Initiate iteration counter
  iterations = 0

  # Containers to hold f1_scores and # of pseudo-labels
  train_f1s = []
  pseudo_labels = []

  # Assign value to initiate while loop
  high_prob = [1] 

  # Loop will run until there are no more high-probability pseudo-labels
  while len(high_prob) > 0:
        
    # Fit classifier and make train/test predictions
    model3.fit(X_train, y_train)
    y_hat_train = model3.predict(X_train)

    # Calculate and print iteration # and f1 scores, and store f1 scores
    train_f1 = f1_score(y_train, y_hat_train)
    print(f"Iteration {iterations}")
    print(f"Train f1: {train_f1}")
    train_f1s.append(train_f1)
   
    if (len(X_unlabeled) > 0):
      # Generate predictions and probabilities for unlabeled data
      print(f"Now predicting labels for unlabeled data...")

      pred_probs : np.ndarray = model3.predict_proba(X_unlabeled)
      preds : np.ndarray = model3.predict(X_unlabeled)
      prob_0 : list = pred_probs[:,0]
      prob_1 : list = pred_probs[:,1]

      # Store predictions and probabilities in dataframe
      df_pred_prob : pd.DataFrame = pd.DataFrame([])
      df_pred_prob['preds'] = preds
      df_pred_prob['prob_0'] = prob_0
      df_pred_prob['prob_1'] = prob_1
      df_pred_prob.index = X_unlabeled.index
    
      # Separate predictions with > 60% probability
      high_prob : pd.DataFrame = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.50],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > 0.50]],
                          axis=0)
      print(f"{len(high_prob)} high-probability predictions added to training data.")
    
      pseudo_labels.append(len(high_prob))

      # Add pseudo-labeled data to training data
      X_train = pd.concat([X_train, X_unlabeled.loc[high_prob.index]], axis=0)
      y_train = pd.concat([y_train, high_prob.preds])      
    
      # Drop pseudo-labeled instances from unlabeled data
      X_unlabeled = X_unlabeled.drop(index=high_prob.index)
      print(f"{len(X_unlabeled)} unlabeled instances remaining.\n")
    
      # Update iteration counter
      iterations += 1
    else :
      high_prob = []
      print(f'end of process.')

  X_train['prediction_supervise'] = y_train
  X_train['index'] = X_train.index
  df["index"] = df.index
  df_final = pd.merge(left=df, right=X_train, left_on='index', right_on='index')
  df_final = df_final[['art_id','prediction_supervise_y']]
  df_final = df_final.rename(columns = {'prediction_supervise_y': 'prediction_supervise'})

  return df_final

In [ ]:
#Function that compute score of innovation and the score of gestion.
#To make this score we run a cross validation and predict the proba on the test set. The proba on the test set is used to make the score of
#innovation and the score of gestion.
def compute_score(data: pd.DataFrame):
  """Documentation
  Parameters:
    data: Data used to compute the score of innovation and the score of gamme gestion

  Out:
    data: DataFrame which contain the id, the score of innovation and the score of gamme gestion
  """
  data.reset_index(drop=True,inplace = True)

  #Creation of the kfold for split train and test
  skf = StratifiedKFold(n_splits=5, random_state = 42, shuffle=True)
  data["score_innovant"] = 0
  data["score_gamme_gestion"] = 0

  #We create cross validation to predict the proba on test set (and use this proba as score)
  for train_index, test_index in skf.split(data, data["innovant"]):
    #Initialisation of the model
    model = RandomForestClassifier(random_state=42)
    #We train on train set
    model.fit(data.drop(["innovant","gamme_gestion","art_id","score_innovant","score_gamme_gestion"],axis = 1).loc[train_index],data.loc[train_index,"innovant"])
    #We predict on test set and we use the proba make the decision score
    data.loc[test_index,"score_innovant"] = model.predict_proba(data.drop(["innovant","gamme_gestion","art_id","score_innovant","score_gamme_gestion"],axis = 1).loc[test_index])[:,1]

  #We create cross validation to predict the proba on test set (and use this proba as score)
  for train_index, test_index in skf.split(data, data["innovant"]):
    #Initialisation of the model
    model = RandomForestClassifier(random_state=42)
    #We train on train set
    model.fit(data.drop(["innovant","gamme_gestion","art_id","score_innovant","score_gamme_gestion"],axis = 1).loc[train_index],data.loc[train_index,"gamme_gestion"])
    #We predict on test set and we use the proba make the decision score
    data.loc[test_index,"score_gamme_gestion"] = model.predict_proba(data.drop(["innovant","gamme_gestion","art_id","score_innovant","score_gamme_gestion"],axis = 1).loc[test_index])[:,1]

  #We round the score of innovation and gamme gestion
  data["score_innovant"] = np.round(data["score_innovant"],3)
  data["score_gamme_gestion"] = np.round(data["score_gamme_gestion"],3)

  return data[["art_id","score_innovant","score_gamme_gestion"]]

In [ ]:
#Function that train the model
def train_model(data: pd.DataFrame, path_to: str):
  '''Documentation
  Parameters:
        data : data to train the model
        data_path : path to save data
  '''
  #Initialisation of the model
  model = RandomForestClassifier(random_state=42)
  #Train model
  model.fit(data.drop(["innovant","gamme_gestion"], axis = 1),data["innovant"])
  #Save the model
  pickle.dump(model, open(path_to + "model_innovant.pkl", 'wb'))

  #Initialisation of the model
  model = RandomForestClassifier(random_state=42)
  #Train model
  model.fit(data.drop(["innovant","gamme_gestion"], axis = 1),data["gamme_gestion"])
  #Save the model
  pickle.dump(model, open(path_to + "model_gamme_gestion.pkl", 'wb'))

# Detection New Documents

In [ ]:
#Function who take a dataframe, normalize the data, run a tsne on 3 components and run an isolation forest.
#The purpose of this function is to predict the new documents
def duplicate_prediction(data: pd.DataFrame, path_save: str) -> pd.DataFrame:
    """Documentation
    Parameters:
        data_path : path to data
    
    Out:
        prediction : list of prediction (-1 : News, 1: Common)
    """
    #We create the dataframe that will save the data
    dataout = pd.DataFrame(data["art_id"].values,columns = ["art_id"])
    data = data.drop(["art_id"],axis = 1)

    #We normalize the data
    sc: StandardScaler = StandardScaler()
    X: np.ndarray = sc.fit_transform(data)
    pickle.dump(sc, open(path_save + "scaler.pkl", 'wb'))

    #We run TSNE
    tsne: TSNE = TSNE(n_components=3)
    X: np.ndarray = tsne.fit_transform(X)
    pickle.dump(tsne, open(path_save + "tsne.pkl", 'wb'))

    #We run isolation forest
    clf: IsolationForest = IsolationForest(random_state=0, contamination=0.01)
    dataout["prediction_nouveau"]: np.ndarray = clf.fit_predict(X)
    dataout["score_nouveau"]: np.ndarray = clf.decision_function(X)
    pickle.dump(clf, open(path_save + "model_novelty.pkl", 'wb'))

    return dataout

# Détection de clusters

In [ ]:
# Find the optimal number of clusters, k
def optimal_cluster(X: pd.DataFrame)->int:
  """Documentation
  Parameters:
      X: dataset containint all our datas we want to train the model with

  Out:
      (list_k[np.argmax(list_error_silhouette)]): optimal value of clusters, k
  """
  #We create a list to stock the silhouette
  list_error_silhouette:list = []
  #We create a list to stock the k
  list_k:list = []
  #The range of k
  K:range = range(9,26)
  #We create the standart scaler to normalize the data
  sc: StandardScaler = StandardScaler()
  X: np.ndarray = sc.fit_transform(X)

  #create the range of k which are going to be used in our loop
  for k in tqdm_notebook(K):
    #We compute pca
    pca: PCA = PCA(n_components=10,random_state = 1)
    #We run kmenoid with k clusters
    km = KMedoids(n_clusters=k,metric='cosine',  random_state=0)
    y = km.fit_predict(pca.fit_transform(X))
    list_k.append(k)
    #create a list with all k values
    list_error_silhouette.append(silhouette_score(X,y))
    #return the maximum of the error based on the silhouette corresponding to the optimal value of k

  return (list_k[np.argmax(list_error_silhouette)])


# Train the chosen model on X matrix with the chosen number of clusters (nb_cluster) and return the trained model for later use
def training(X: pd.DataFrame, nb_cluster: int, path_save: str)->(KMedoids, PCA, StandardScaler):
  """Documentation
  Parameters:
      X: The data to predict
      nb_cluster: The number of cluster
      path_save: The path to save the model

  Out:
      km: The model
      pca: The pca
      sc: The normalisation
  """
  #Nomalisation
  sc: StandardScaler = StandardScaler()
  X: np.ndarray = sc.fit_transform(X)
  pickle.dump(sc, open(path_save + "scaler_cluster.pkl", 'wb'))

  #PCA
  pca: PCA = PCA(n_components=10,random_state = 1)
  X: np.ndarray = pca.fit_transform(X)
  pickle.dump(pca, open(path_save + "pca_cluster.pkl", 'wb'))

  #Kmenoids
  km: KMedoids = KMedoids(n_clusters=nb_cluster,metric='cosine',  random_state=0).fit(X)
  pickle.dump(km, open(path_save + "model_cluster.pkl", 'wb'))

  return km, pca, sc

#We take a dataframe and we return prediction
def predict(X: pd.DataFrame,model: KMedoids,pca: PCA,sc: StandardScaler)->list:
  """Documentation
  Parameters:
      X: The data to predict
      km: The model
      pca: The pca to make before to run the model
      sc: The normalisation to make before to run the pca

  Out:
      return prediction
  """
  #Normalisation
  X: np.ndarray = sc.transform(X)
  #pca
  X: np.ndarray = pca.transform(X)
  
  #prediction
  return model.predict(X)

In [ ]:
#calculate the chosen distance (euclidean or cosine) between the values inside
#the cluster with label denomination and the centroid of this cluster.
def compute_distance(v: np.ndarray,km: KMedoids,label: int,pca: PCA,sc: StandardScaler)->list:
  """Documentation
  Parameters:
      v: The article to compute
      km: The model
      label: The number of the cluster
      pca: The pca to make before to run the model
      sc: The normalisation to make before to run the pca

  Out:
      distance bewteen the point and the center of the cluster
  """
  #We make the euclidian distance between the point and the center of the cluster
  return distance.euclidean(pca.transform(sc.transform(v.reshape(1, -1))),km.cluster_centers_[label])

#We give a list of article and return the 3 most recurents word
def obtain_word(list_content: list, type = "idf")-> str:
  """Documentation
  Parameters:
      X: contains the data

  Out:
      (list_k[np.argmax(list_error_silhouette)]): optimal value of clusters, k
  """
  if type == "idf":
    #We drop english word because there is some english word in the content
    vectorizer = TfidfVectorizer(stop_words = "english")
    X = vectorizer.fit_transform(list_content)
    #Vectorizes all the tags in the train set while cleaning it from stop words
  elif type == "bow":
    #We drop english word because there is some english word in the content
    vectorizer = CountVectorizer(stop_words = "english")
    X = vectorizer.fit_transform(list_content)
    #Vectorizes all the tags in the train set while cleaning it from stop words
  else:
    print("None")
    #return an error while the type specified is not idf or bow

  #We create a dataframe with the sum of the bow/td-idf and we take the 3 word with the biggest score
  temp_data = pd.DataFrame(np.array(np.sum(X,axis = 0))[0], columns = ["score"])
  temp_data["word"] = vectorizer.get_feature_names()
  #Create a word column from feature integer indices to feature name.
  temp_data = temp_data.sort_values(by = ["score"], ascending = False)
  return temp_data.iloc[0,1] + "/" + temp_data.iloc[1,1] + "/" + temp_data.iloc[2,1]


# Get the content of an article except the words present in the whitelist
def whitelist_content(list_content: list, list_whitelist: list) -> np.ndarray:
  """Documentation
  Parameters:
      list_content: A list of articles
      list_whitelist: A whitelist of word

  Out:
      final_content: A list of articles without the word in the whitelist
  """
  #We create a list
  final_content = []

  for content in list_content:
    #We take every word witch are not present in whitelist
    temp_content = [w for w in content.split() if not w in list_whitelist]
    #We join all the word to create a sentence
    temp_content = " ".join(temp_content)
    final_content.append(temp_content)

  return np.array(final_content)


#We compute the distance between the centroide of each cluster and all the point of this cluster
#we take the 5 closest point of the cluster and we use this 5 articles to select the name of the cluster.
#To select the name of the cluster we take the 3 most recurrents words.
def get_label_title(data: pd.DataFrame,whitelist: list,km: KMedoids,pca: PCA,sc: StandardScaler, path_save: str) -> dict:
  """Documentation
  Parameters:
      data: A dataframe containing all our datas we want to work with
      whitelist: a whitelist of words we want to avoid in labels title

  Out:
      themes: A string containing our title of the chosen label
  """
  #We create the dictionnary that will contain the label corresponding to each cluster
  themes = {}

  for label in np.unique(data["prediction"]):
    #We create a copy of the dataframe which only contain the cluster corresponding to the value of label
    sub_data = data.query("prediction == @label").drop(["prediction"],axis = 1).copy()
    #We compute the distance bewteen the center of the cluster and all the point of the cluster
    sub_data["score"] = sub_data.drop(["art_id","art_content_clean_without_lem"], axis = 1).apply(compute_distance,axis = 1,raw = True, args = (km,0,pca,sc))
    #We take the five closest document of the center
    head_content = sub_data.sort_values(by = ["score"], ascending = True).head(5)["art_content_clean_without_lem"].values
    #We drop the word in the whitelist
    head_content = whitelist_content(head_content,whitelist)
    #We get the label of ech cluster by checking the three recurents words
    themes[label] = obtain_word(head_content, type = "bow")
    #We save the label in a dictionnary
    pickle.dump(themes, open(path_save + "dict_label.pkl", 'wb'))
    
  return themes

In [ ]:
# Clusters the dataset with the optimal number of clusters and return
def predict_cluster(data: pd.DataFrame, path_save: str)-> pd.DataFrame:
  """Documentation
  Parameters:
      data: A dataframe containing all our datas we want to work with
      
  Out:
      data[["art_id","prediction"]]: A dataframe containing our id and the title of the chosen label
      model: the model trained with the optimal number of clusters
  """
  #We use the function higher to obtain the good number of clusters
  nb_cluster = optimal_cluster(data.drop(["art_id","art_content_clean_without_lem"], axis = 1))
  print("We take " + str(nb_cluster) + " clusters.")
  #We train the kmenoide, pca and normalisation to make prediction
  model, pca, sc = training(data.drop(["art_id","art_content_clean_without_lem"], axis = 1),nb_cluster, path_save)
  #We predict the cluster with the kmenoide, pca and normalisation trained
  data["prediction"] = predict(data.drop(["art_id","art_content_clean_without_lem"], axis = 1),model, pca, sc)
  #We create a whitelist to drop some recurents word that was not drop
  whitelist = ["plus","tout","cette","ca","etre","dire","faut","fait","faire","donc","avoir","être","leur","le","mettre","pouvoir","art_id","entreprise","gestion","entreprises","aussi","meme","bien","ete","comme","etat","ministre","tres","encore","peut","dont","egalement","notamment","ainsi","leurs","entre"]
  #We create the label of each cluster
  data["name_theme"] = data["prediction"].map(get_label_title(data,whitelist,model,pca,sc,path_save))

  return data[["art_id","name_theme"]]

#Big function wich call all the other function

In [ ]:
def all_prediction(data: pd.DataFrame,lexique_innovant: pd.DataFrame,lexique_gestion: pd.DataFrame,path_save: str) -> pd.DataFrame:
  """Documentation
  Parameters:
      data: A dataframe containing 3 columns(art_id, art_content, art_uitle) and the bow
  Output:
      data: A dataframe with all the the predicted features
  """
  np.random.seed(42)
  #We compute innovant and gamme gestion
  print("Phase 1")
  #We predict the label innovant
  data_temp_innovant = create_label(data.copy(),"innovation",lexique_innovant)
  data_temp_innovant.columns = ["art_id","innovant"]
  #We predict the label gestion
  data_temp_gestion = create_label(data.copy(),"gestion",lexique_gestion)
  data_temp_gestion.columns = ["art_id","gamme_gestion"]
  #We make a concatenation bewteen the input data and the predicted label
  data = data.merge(data_temp_innovant, on = "art_id").merge(data_temp_gestion, on = "art_id")
  data.rename(columns = {"innovant_y" : "innovant","gamme_gestion_y" : "gamme_gestion"}, inplace = True)

  #We compute the score
  print("Phase 2")
  #We predict the score of innovant and gamme gestion
  data_temp = compute_score(data.drop(["art_content_clean_without_lem","art_title"], axis = 1).copy())
  data = data.merge(data_temp, on = "art_id")
  #We make a concatenation bewteen the input data and the predicted label
  data.rename(columns = {"score_innovant_y" : "score_innovant","score_gamme_gestion_y" : "score_gamme_gestion"}, inplace = True)
  #We train model
  train_model(data.drop(["art_id","art_content_clean_without_lem","score_innovant","score_gamme_gestion","art_title"], axis = 1).copy(),path_save)

  #We compute the new documents
  print("Phase 3")
  #We predict the new document
  data_temp = duplicate_prediction(data.query("innovant == 1").query("gamme_gestion == 1").drop(["art_content_clean_without_lem","art_title","score_innovant","score_gamme_gestion","innovant","gamme_gestion"], axis = 1).copy(),path_save)
  #We make a concatenation bewteen the input data and the predicted label
  data_temp["prediction_nouveau"] = data_temp["prediction_nouveau"].replace([1,-1],[0,1])
  data = data.merge(data_temp,how = "left", on = "art_id")
  data.rename(columns = {"score_nouveau" : "score_nouveau"}, inplace = True)

  #On predict the cluster
  print("Phase 4")
  #We run the function who predict the number of cluster
  data_temp = predict_cluster(data.query("innovant == 1").query("gamme_gestion == 1").drop(["innovation","gamme_gestion","score_innovant","score_gamme_gestion","prediction_nouveau","score_nouveau","art_title"], axis = 1).copy(),path_save)
  #We join the data
  data = data.merge(data_temp,how = "left", on = "art_id")
  #We select the columns that we want to give in output 
  data = data[["art_id","innovant","gamme_gestion","score_innovant","score_gamme_gestion","prediction_nouveau","score_nouveau","name_theme"]].copy()
  #We rename some columns
  data.rename(columns = {"name_theme" : "theme","prediction_nouveau" : "nouveau"}, inplace = True)

  return data

# We run the function

In [ ]:
#We load the data who contain the BOW
data = pd.read_json(
    "/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/g3_BOW_v1.json")

In [ ]:
#We load the data who contain the title
data_title = pd.read_csv(
    '/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/Data_With_Features_Syntax.csv')

In [ ]:
#We load the data whith lemmatization on the content
data_lemma = pd.read_json(
    '/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Output/Innovation/df_articles_lemma.json')

In [ ]:
#We load the lexique of innovation
df_lexique_innovation: pd.DataFrame = pd.read_json(
    "/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Output/Innovation/df_lexique_lemma.json")

In [ ]:
#We load the lexique of gestion
df_lexique_gestion: pd.DataFrame = pd.read_json(
    "/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Output/Innovation/df_lexique_gammes_gestion.json")

In [ ]:
#We concatene all the data
data["art_title"] = data_title.loc[data.index,"art_title"]
data["art_content_clean_without_lem"] = data_lemma["art_lemma"]
del(data_title)
del(data_lemma)

In [ ]:
#We run the function who make all the prediction
output = all_prediction(data,df_lexique_innovation,df_lexique_gestion,"/content/drive/MyDrive/G5 Inter-Promo 2021/Ressources/test/")

In [ ]:
#We save the data
output.to_json("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Output/Global/Global_V2.json",orient="records")

In [ ]:
output.shape